In [4]:
import requests
import pandas as pd
import dash
from dash import dcc
from dash import html
import plotly.express as px
from io import StringIO
import plotly.graph_objs as go

In [5]:
def fetch_data(endpoint):
    url = f"http://apps.who.int/gho/athena/api/GHO/{endpoint}?format=csv"
    response = requests.get(url)
    print(f'Fetched data for {endpoint}')
    csv_data = StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(csv_data)
    df = keep_most_recent_date(df).loc[:, ['COUNTRY', 'YEAR', 'Numeric','REGION']]
    return df

def keep_most_recent_date(df):
    df = df.sort_values(by=['COUNTRY', 'YEAR'], ascending=[True, False])
    df = df.drop_duplicates(subset='COUNTRY', keep='first')
    return df

In [6]:
stunting = "NUTRITION_ANT_HAZ_NE2"

In [7]:
df = fetch_data(stunting)


Fetched data for NUTRITION_ANT_HAZ_NE2


In [43]:
df[df['REGION'] == most_affected_region].groupby('YEAR').count()

,COUNTRY,Numeric,REGION
YEAR,,,
1994,1,1,1
1995,1,1,1
2007,1,1,1
2010,2,2,2
2011,1,1,1
2012,3,3,3
2013,1,1,1
2014,3,3,3
2015,1,1,1


In [36]:
grouped_df = df.groupby('REGION')['Numeric'].mean().reset_index()
max_index = grouped_df['Numeric'].idxmax()
most_affected_region = grouped_df.loc[max_index,'REGION']
df['YEAR'] = df['YEAR'].astype(int)
df[df['REGION'] == most_affected_region].groupby('YEAR')['Numeric'].mean().reset_index(drop=False).sort_values(by="YEAR")

,YEAR,Numeric
0,1994,21.400000
1,1995,13.600000
2,2007,28.900000
3,2010,41.900000
4,2011,26.200000
5,2012,18.666667
6,2013,22.700000
7,2014,24.300000
8,2015,37.600000
9,2016,21.600000
